# Mandatory Challenge
## Context
You work in the data analysis team of a very important company. On Monday, the company shares some good news with you: you just got hired by a major retail company! So, let's get prepared for a huge amount of work!

Then you get to work with your team and define the following tasks to perform:   
1. You need to start your analysis using data from the past.  
2. You need to define a process that takes your daily data as an input and integrates it.  

You are in charge of the second part, so you are provided with a sample file that you will have to read daily. To complete you task, you need the following aggregates:
* One aggregate per store that adds up the rest of the values.
* One aggregate per item that adds up the rest of the values.

You can import the `raw_sales` table from the database `retail_sales` fon of Ironhack's databases. 

## Your task
Therefore, your process will consist of the following steps:
1. Read the sample file that a daily process will save in your folder. 
2. Clean up the data.
3. Create the aggregates.
4. Write three tables in your local database: 
    - A table for the cleaned data.
    - A table for the aggregate per store.
    - A table for the aggregate per item.

## Instructions
* Clean the data and create the aggregates as you consider.
* Create the tables in your local database.
* Populate them with your process.

In [1]:
# Import external libraries
import sqlalchemy
import pandas as pd

In [2]:
# Configure SQL connection
driver   = 'mysql+pymysql:'
user     = 'ironhack'
password = ''
host     = 'localhost'
database = 'retail_sales'

In [3]:
# Test the connection & create the connection
connection_string = f'{driver}//{user}:{password}@{host}/{database}'
print(connection_string)
engine = sqlalchemy.create_engine(connection_string)

mysql+pymysql://ironhack:@localhost/retail_sales


# Task 1 - Read the Data

In [4]:
df = pd.read_sql('SELECT * FROM raw_sales', engine)
df.head()

,date,shop_id,item_id,item_price,item_cnt_day
0,2015-01-04,29,1469,1199.0,1.0
1,2015-01-04,28,21364,479.0,1.0
2,2015-01-04,28,21365,999.0,2.0
3,2015-01-04,28,22104,249.0,2.0
4,2015-01-04,28,22091,179.0,1.0


# Task 2 - Clean Up the Data

In [5]:
# Find if any column has NaN value
df.isna().sum()

date            0
shop_id         0
item_id         0
item_price      0
item_cnt_day    0
dtype: int64

In [6]:
# Find if any column has Null value
df.isnull().sum()

date            0
shop_id         0
item_id         0
item_price      0
item_cnt_day    0
dtype: int64

In [7]:
# Get info about the DataFrame
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4545 entries, 0 to 4544
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   date          4545 non-null   datetime64[ns]
 1   shop_id       4545 non-null   int64         
 2   item_id       4545 non-null   int64         
 3   item_price    4545 non-null   float64       
 4   item_cnt_day  4545 non-null   float64       
dtypes: datetime64[ns](1), float64(2), int64(2)
memory usage: 177.7 KB


# Task 3 - Create the aggregates

In [9]:
aggStore = df.groupby('shop_id').agg({'item_price':['sum','mean','min','max','count']})
aggStore.head()

item_price                                  
               sum         mean    min     max count
shop_id                                             
2          99070.5  1320.940000   28.0  8999.0    75
3          67443.0  2043.727273  500.0  8999.0    33
4          29361.0   752.846154   79.0  2799.0    39
5          33138.0   736.400000   99.0  3690.0    45
6         116352.0   923.428571    5.0  3999.0   126

In [10]:
aggItem = df.groupby('item_id').agg({'item_price':['sum','mean','min','max','count'],'item_cnt_day':['sum']})
aggItem

item_price                            item_cnt_day
               sum   mean    min    max count          sum
item_id                                                   
30           507.0  169.0  169.0  169.0     3          3.0
31          1089.0  363.0  363.0  363.0     3          3.0
32           447.0  149.0  149.0  149.0     3          3.0
42           897.0  299.0  299.0  299.0     3          3.0
59           747.0  249.0  249.0  249.0     3          3.0
...            ...    ...    ...    ...   ...          ...
22091       1074.0  179.0  179.0  179.0     6          6.0
22092        537.0  179.0  179.0  179.0     3          3.0
22104        747.0  249.0  249.0  249.0     3          6.0
22140        652.5  217.5  217.5  217.5     3          3.0
22162       5985.0  399.0  399.0  399.0    15         18.0

[985 rows x 6 columns]

In [13]:
df.to_sql(con=engine, name='cleaned_data', if_exists='replace')

In [14]:
aggItem.to_sql(con=engine, name='agg_per_item', if_exists='replace')

In [15]:
aggStore.to_sql(con=engine, name='agg_per_store', if_exists='replace')